In [4]:
%matplotlib inline
import torch 
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [5]:
import mnist_loader
(train_images, train_labels, test_images, test_labels) = mnist_loader.load_fashion_mnist()

data folder already exists
Loading training images...
Loading training labels...
Loading testing images...
Loading testing labels...


In [6]:
from utils import get_one_hot, DataSet
train_labels = get_one_hot(train_labels)
test_labels = get_one_hot(test_labels)

In [7]:
train_images = train_images.reshape((-1, 1, 28, 28)).astype(np.float32)
test_images = test_images.reshape((-1, 1, 28, 28)).astype(np.float32)
train_labels = train_labels.astype(np.int64)
test_labels = test_labels.astype(np.int64)
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


In [ ]:
from torch.utils.data import Dataset, DataLoader

train_dataset = DataSet(test_images, test_labels)
test_dataset = DataSet(train_images, train_labels)
    
test_loader = DataLoader(train_dataset, num_workers=1, batch_size=10000)
train_loader = DataLoader(test_dataset, shuffle=True, batch_size=1000)



In [ ]:
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
              7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'};

In [ ]:
#source: https://medium.com/ml2vec/intro-to-pytorch-with-image-classification-on-a-fashion-clothes-dataset-e589682df0c5
fig = plt.figure(figsize=(8,8));
columns = 4;
rows = 5;

for i in range(1, columns*rows +1):
    img_xy = np.random.randint(len(train_dataset));
    img = train_dataset[img_xy][0][0,:,:]
    fig.add_subplot(rows, columns, i)
    plt.title(labels_map[np.argmax(train_dataset[img_xy][1])])
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

In [2]:
network_dense_config = {
    'mode': 'dense',
    'units': [512],
    'dropouts': [0.2],
}

dense_net = net.Network(
    name='3_dense_test',
    dimensions=[None] + list(train_images.shape[1:]),
    input_var=input_var,
    y=y,
    config=network_dense_config,
    input_network=None,
    num_classes=10,
    activation='rectify',
    pred_activation='softmax',
    optimizer='adam')


NameError: name 'train_images' is not defined

In [3]:
train_images.shape

NameError: name 'train_images' is not defined

In [ ]:
network_conv_config = {
    'mode': 'conv',
    'filters': [16, 32],
    'filter_size': [[5, 5], [5, 5]],
    'stride': [[1, 1], [1, 1]],
    'pool': {
        'mode': 'average_exc_pad',
        'stride': [[2, 2], [2, 2]]
    }
}
network_dense_config = {
    'mode': 'dense',
    'units': [512],
    'dropouts': [0.3],
}

In [ ]:
import net

In [ ]:
#input_var = Variable(train_images.shape, requires_grad=True)
#y = Variable(train_labels.shape, requires_grad=True)
conv_net= net.Network(
    name='conv_test',
    dimensions= train_images.shape,
    input_var=None,
    y=None,
    config=network_conv_config,
    input_network=None,
    num_classes=None)

In [ ]:
dense_net = net.Network(
    name='1_dense',
    dimensions=(None, int(train_images.shape[1])),
    input_var=0,
    y=0,
    config=network_dense_config,
    input_network={'network': conv_net, 'layer': 4, 'get_params': True},
    num_classes=10,
    activation='rectify',
    pred_activation='softmax'
)

In [ ]:
dense_net.network

In [ ]:
dense_net.train(
    epochs=200,
    train_x=train_images[:50000],
    train_y=train_labels[:50000],
    val_x=train_images[50000:60000],
    val_y=train_labels[50000:60000],
    batch_ratio=0.05,
    plot=False,
    use_gpu=True
)

In [ ]:
for param in dense_net.network.parameters():
    print(param)

In [ ]:
import collections

In [ ]:

mods = list(dense_net.network.modules())
for i in range(1,len(mods)):
    m = mods[i]
    p = list(m.parameters())
    sizes = []
    print(m)
    for j in range(len(p)):
        size = np.array(p[j].size())
        sizes.append(size)
        print(size)

total_bits = 0
for i in range(len(sizes)):
    s = sizes[i]
    bits = np.prod(np.array(s))*bits
    print(bits)
    total_bits += bits

In [ ]:
for l_name, l in dense_net.network.named_children():
    if isinstance(l, torch.nn.Sequential):
        for subl_name, subl in l.named_children():
            for param in subl.parameters():
                print(l_name, subl_name, param.size(0))
    else:
        for param in l.parameters():
            print(l_name, param.size(0))


In [ ]:
torch.cuda.is_available()

In [ ]:
model = dense_net.network

optimizer = torch.optim.Adam(dense_net.network.parameters(), lr=0.01);

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [ ]:
import os
os.getpid()

In [ ]:
losses = []
epochs = 100
optimizer = torch.optim.Adam(dense_net.network.parameters(), lr=0.01);
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss_fn = nn.CrossEntropyLoss()
        loss = criterion(outputs, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data[0]);
       
        if (i+1) % 100 == 1:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, epochs, i+1, len(train_dataset)//100, loss.data[0]))

In [ ]:
isinstance(model, nn.Module)

In [ ]:
import tqdm

In [ ]:
dense_net.network._optimizer.state_dict()

In [ ]:
def train(network, epochs, train_loader, test_loader,
              plot=True, change_rate=None, use_gpu=False):
    
    print('\nTraining {} in progress...\n'.format(self.name))
    # Check if to use_gpu
    self.use_gpu = use_gpu

    

    if train_x.shape[0] * batch_ratio < 1.0:
        batch_ratio = 1.0 / train_x.shape[0]
        print('Warning: Batch ratio too small. Changing to {:.5f}'.format(batch_ratio))
    try:
        if plot:
            fig_number = plt.gcf().number + 1 if plt.fignum_exists(1) else 1

        self.optimizer = torch.optim.SGD(self.network.parameters(),
                                            lr=0.01, momentum=0.5)
        self.criterion = nn.CrossEntropyLoss()

        if self.use_gpu:
            self.network = self.network.cuda()
            self.criterion = self.criterion.cuda()

        for epoch in range(epochs):
            epoch_time = time.time()
            print("--> Epoch: {}/{}".format(
                epoch,
                epochs - 1
            ))
            for mode in ['train', 'val']:
                if mode == 'train':
                    self.network.train()  # Set model to training mode
                    losses = []
                    loader = tqdm(train_loader, total=len(train_loader))
                    for batch_idx, (data, target) in enumerate(loader):

                        data = Variable(data.float())
                        target = Variable(target)

                        if self.use_gpu:
                            data = data.cuda()
                            target = target.cuda()

                        # Forward + Backward + Optimize
                        self.optimizer.zero_grad()
                        output = self.network(data)
                        loss = nn.CrossEntropyLoss()(output, torch.max(target, 1)[1])
                        loss.backward()
                        self.optimizer.step()
                        losses.append(float(loss.item()))
                    print('\tLoss: {:.6f} {}'.format(loss.item(), loss.loss_tot.data[0]))

                else:
                    self.network.eval()  # Set model to evaluate mode
                    test_loss = 0
                    correct = 0
                    with torch.no_grad():
                        for data, target in test_loader:

                            data = Variable(data.float())
                            target = Variable(target)

                            if torch.cuda.is_available():
                                data = data.cuda()
                                target = target.cuda()

                            output = self.network(data)
                            test_loss += nn.CrossEntropyLoss()(output, torch.max(target, 1)[1])
                            pred = output.max(1)[1] # get the index of the max log-probability
                            correct += pred.eq(torch.max(target, 1)[1]).sum().item()

                    test_loss /= len(test_loader.dataset)
                    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                        test_loss, correct, len(test_loader.dataset),
                        100. * correct / len(test_loader.dataset)))
    except KeyboardInterrupt:
        print("\n\n**********Training stopped prematurely.**********\n\n")
    finally:
        self.timestamp = get_timestamp()